In [12]:
from openai import OpenAI
import pandas as pd

In [23]:
# Load the dataset
df = pd.read_csv("BBC_News_Summary.csv")

In [30]:
# Set OpenAI API key
openai.api_key = 'your_openai_api_key_here'

client = OpenAI(api_key = openai.api_key)
# Function to classify bias using ChatGPT
def detect_bias(text):
    prompt = (
        "Please classify the following article into one of five bias classes: "
        "'far left', 'left', 'neutral', 'right', 'far right'. "
        "Use these values for classification: far left: -2, left: -1, neutral: 0, right: 1, far right: 2.\n\n"
        f"Article:\n{text}\n\n"
        "Return your result in the format: 'Bias: <class>, Value: <number>'"
    )
    
    # Call ChatGPT
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Extract bias classification from the response
    content = response.choices[0].message.content
    
    try:
        bias_class = content.split("Bias: ")[1].split(",")[0].strip()
        bias_value = int(content.split("Value: ")[1].strip())
    except:
        bias_class = "unknown"
        bias_value = 9
    
    return bias_class, bias_value
    

In [ ]:
# Apply the function to detect bias in articles and summaries
df[['article bias', 'article bias value']] = df['Article'].apply(lambda x: detect_bias(x)).apply(pd.Series)
df[['summary bias', 'summary bias value']] = df['Summary'].apply(lambda x: detect_bias(x)).apply(pd.Series)

In [ ]:
# Analyze bias deviation
def analyze_bias_deviation(article_bias_value, summary_bias_value):
    if summary_bias_value > article_bias_value:
        return "exaggerate"
    elif summary_bias_value < article_bias_value:
        return "mitigate"
    else:
        return "preserve"

df['bias deviation'] = df.apply(
    lambda row: analyze_bias_deviation(row['article bias value'], row['summary bias value']), axis=1)


In [ ]:
# Calculate the directional bias deviation
df['bias deviation (directional)'] = df.apply(
    lambda row: row['summary bias value'] - row['article bias value'], axis=1)

In [ ]:
# Save the results to a new CSV file
df.to_csv("BBC_News_Bias_Analysis.csv", index=False, encoding='utf-8')

print("Bias detection and analysis completed. Results saved to 'BBC_News_Bias_Analysis.csv'.")

In [ ]:
# plot results
import matplotlib.pyplot as plt

# Make sure df is your DataFrame with 'summary bias value' and 'article bias value'
# Example: df = pd.read_csv('your_data.csv')

# Plotting the distribution of 'summary bias value' and 'article bias value'
plt.figure(figsize=(10, 6))
plt.hist(df['summary bias value'], bins=5, alpha=0.5, label='Summary Bias Value', color='blue')
plt.hist(df['article bias value'], bins=5, alpha=0.5, label='Article Bias Value', color='orange')
plt.xlabel('Bias Value')
plt.ylabel('Frequency')
plt.title('Distribution of Summary Bias Value and Article Bias Value')
plt.legend(loc='upper right')
plt.show()